In [ ]:
# 指定各模型保存目录
base_model_dir = "../TockFormerModels/BaseModels"
scaler_dir     = "../TockFormerModels/Scalers"
blender_dir    = "../TockFormerModels/Blenders"


# 填入你保存时用的日期
date_str = datetime.now().strftime("%Y%m%d")

In [ ]:
# 组装各文件完整路径
transformer_path = os.path.join(base_model_dir, f"Transformer_{date_str}.pth")
lstm_path        = os.path.join(base_model_dir, f"LSTM_{date_str}.pth")
ridge_path       = os.path.join(base_model_dir, f"Ridge_{date_str}.pkl")
xgbst_path       = os.path.join(base_model_dir, f"XGBST_MultiOutput_{date_str}.pkl")
scaler_path      = os.path.join(scaler_dir,     f"StandardScaler_{date_str}.pkl")
blender_path     = os.path.join(blender_dir,    f"RidgeBlender_{date_str}.pkl")

In [ ]:
# 检查设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 加载pytorch类
from TransformerModel import TimeSeriesTransformer
from LSTMModel       import TimeSeriesLSTM


# ———— 根据你实际的超参来初始化 ————
tock = TimeSeriesTransformer(
    input_dim=49,
    model_dim=128,
    num_heads=4,
    num_layers=2,
    dropout=0.1,
    seq_length=60,
    output_dim=4
)
tock.load_state_dict(torch.load(transformer_path, map_location=device))


lstm = TimeSeriesLSTM(
    input_size=49,
    hidden_size=128,
    num_layers=2,
    output_size=4
)
lstm.load_state_dict(torch.load(lstm_path, map_location=device))

In [ ]:
# 加载 sklearn/XGBoost 模型与 Scaler
ridge         = joblib.load(ridge_path)
xgbst_mlt     = joblib.load(xgbst_path)
scaler        = joblib.load(scaler_path)
ridge_blender = joblib.load(blender_path)

In [ ]:
import numpy as np
import pandas as pd

# 数据参数
seq_len = 32
target_cols = ["open", "high", "low", "close"]

# 读取数据
# df = pd.load_csv("xxxxx.csv")

X, y, _, _ = create_sequences(df, seq_length=seq_len, target_cols=target_cols, scaler=scaler)

In [ ]:

# 各基模型预测
pred_tock = tock.predict_model()
pred_lstm = lstem.predict_model()

pred_ridge = ridge.predict(X_scaled)
pred_xgb   = xgbst_mlt.predict(X_scaled)

In [ ]:
# 拼接做 Blender
# 这里假设你有4个基模型的预测结果，形状 (n_models, model_output_dim)
blend_input = np.concatenate([
    pred_tock,
    pred_lstm,
    pred_ridge,
    pred_xgb
], axis=1)

blend_pred = ridge_blender.predict(blend_input)

print("Blended 预测结果示例：")
print(blend_pred[:5])